In [ ]:
import numpy
import pandas as pd
from functools import reduce
from datetime import datetime
import datetime
import datetime
today_dt = pd.to_datetime('20.12.2020', format='%d.%m.%Y')

In [ ]:

# Основная выборка, разбитая на train и test

train = pd.read_csv('Public/train.csv', sep=';')
test = pd.read_csv('Public/test.csv', sep=';')

# Дополнительные данные 

ed = pd.read_csv('Public/education.csv', sep=';')
ws = pd.read_csv('Public/worldskills.csv', sep=';')
em = pd.read_csv('Public/employements.csv', sep=';')
em_multi = pd.read_csv('Public/employements_mult.csv', sep=';')
ed_multi = pd.read_csv('Public/education_mult.csv', sep=';')


train = train.merge(ed, left_on='id', right_on='id', how = 'left')
train = train.merge(em, left_on='id', right_on='id', how = 'left')
#train = train.merge(ws, left_on='id', right_on='id')

In [ ]:
# количество дней на каждом месте
em_multi['start_date'] = em_multi['start_date'].fillna(value = today_dt)
em_multi['start_date'] = pd.to_datetime(em_multi['start_date'], format='%Y-%m-%d', errors = 'coerce')

em_multi['finish_date'] = em_multi['finish_date'].fillna(value = today_dt)
em_multi['finish_date'] = pd.to_datetime(em_multi['finish_date'], format='%Y-%m-%d', errors = 'coerce')
em_multi['days_at_th_work'] = em_multi['finish_date'] - em_multi['start_date']
em_multi['days_at_th_work'] = em_multi['days_at_th_work'].apply(lambda x: x.days)
em_multi['total_work_days'] = em_multi['days_at_th_work']

In [ ]:
#агрегируем. Считаем количество мест, среднее количество дней на одном мес те

em_multi_1 = em_multi[['id', 'employer']].groupby(['id'], as_index=False).agg('count')
em_multi_2 = em_multi[['id', 'days_at_th_work']].groupby(['id'], as_index=False).agg('mean')
em_multi_2a = em_multi[['id', 'total_work_days']].groupby(['id'], as_index=False).agg('sum')
em_multi_3 = em_multi[['id', 'start_date']].groupby(['id'], as_index=False).agg('min')
em_multi_4 = em_multi[['id', 'finish_date']].groupby(['id'], as_index=False).agg('max')
em_multi_ls = [em_multi_1, em_multi_2, em_multi_2a, em_multi_3, em_multi_4]
    
#объединяем все нагрегированные датасеты в один большой
em_multi_gr = reduce(lambda  left,right: pd.merge(left,right,on=['id'], how='right'), em_multi_ls)
em_multi_gr = em_multi_gr.rename(columns = {'employer': 'emp_count', 'finish_date' : 'total_finish_date', 'start_date' :  'total_start_date'})

#могли быть перерывы между работами
#em_multi_gr['total_work_days'] = em_multi_gr['total_finish_date'] - em_multi_gr['total_start_date']
#em_multi_gr['total_work_days'] = em_multi_gr['total_work_days'].apply(lambda x: x.days)

In [ ]:
train = train.merge(em_multi_gr, left_on='id', right_on='id', how='left')
test = test.merge(em_multi_gr, left_on='id', right_on='id', how='left')

In [ ]:
#Образование .сколько, какое, москва ли?
ed_multi['institution'] = ed_multi['institution'].fillna(value="")
ed_multi['institution'] = ed_multi['institution'].apply(lambda x: str(x).lower())
ed_multi['high_ed'] = ed_multi['institution'].apply(lambda x: 1 if (('универ' in x) or ('академ' in x) or ('инстит' in x) or 
                                                              ('высш' in x) or ('консерв' in x)) else 0)
ed_multi['voc_ed'] = ed_multi['institution'].apply(lambda x: 1 if (('учил' in x) or ('техник' in x) or ('колледж' in x) or 
                                                              ('ссуз' in x) or ('пту' in x)) else 0);
ed_multi['postgr_ed'] = ed_multi['institution'].apply(lambda x: 1 if (('аспиран' in x) or ('докторан' in x) or ('интернату' in x)) else 0)
ed_multi['Mosc'] = ed_multi['institution'].apply(lambda x: 1 if ('моск' in x) else 0 )
ed_multi_gr = ed_multi[['id', 'high_ed', 'voc_ed', 'postgr_ed', 'Mosc']].groupby(['id'], as_index=False).agg('sum')



In [ ]:
train = train.merge(ed_multi_gr, left_on='id', right_on='id', how='left')
test = test.merge(ed_multi_gr, left_on='id', right_on='id', how='left')

In [ ]:
def CrDt(data):
#for date
    data['creation_date'] = pd.to_datetime(data['creation_date'], format='%Y-%m-%d')
    data['create_year'] = pd.DatetimeIndex(data['creation_date']).year
    data["create_month"]= pd.DatetimeIndex(data["creation_date"]).month
    data["create_day"] = pd.DatetimeIndex(data["creation_date"]).day
    data["create_dt_half_m"] =  data["create_day"].apply(lambda x : 2 if (x > 15) else 1)    


    data['modification_date'] = pd.to_datetime(data['modification_date'], format='%Y-%m-%d')
    data['modification_year'] = pd.DatetimeIndex(data['modification_date']).year
    data["modification_month"]= pd.DatetimeIndex(data["modification_date"]).month
    data["modification_day"] = pd.DatetimeIndex(data["modification_date"]).day
    data["modification_dt_half_m"] =  data["create_day"].apply(lambda x : 2 if (x > 15) else 1)


    data['publish_date'] = pd.to_datetime(data['publish_date'], format='%Y-%m-%d')
    data['publish_year'] = pd.DatetimeIndex(data['publish_date']).year
    data["publish_month"]= pd.DatetimeIndex(data["publish_date"]).month
    data["publish_day"] = pd.DatetimeIndex(data["publish_date"]).day
    data["publish_dt_half_m"] =  train["publish_day"].apply(lambda x : 2 if (x > 15) else 1)    

    data['creation_date'] = pd.to_datetime(data['creation_date'], format='%Y-%m-%d')
    data['create_year'] = pd.DatetimeIndex(data['creation_date']).year

    return data



In [ ]:
train = CrDt(train)
test = CrDt(test)

In [ ]:
def bin(data):
    data['gender'] = data['gender'].fillna('Женский')
    data['gender'] = data['gender'].apply(lambda x: 0 if x =='Женский' else 1)

    data['relocation_ready'] = data['relocation_ready'].fillna(False)
    data['relocation_ready'] = data['relocation_ready'].astype(int)

    data['retraining_ready'] = data['retraining_ready'].fillna(True)
    data['retraining_ready'] = data['retraining_ready'].astype(int)

    data['travel_ready'] = data['travel_ready'].fillna(-1)
    data['travel_ready'] = data['travel_ready'].astype(int)
    return data

In [ ]:
train = bin(train)
test = bin(test)

In [ ]:
def coronavir_stat(data):
    data = data[['id','creation_date','modification_date','publish_date']].copy()
    data['creation_date'] = pd.to_datetime(data['creation_date'])
    data['modification_date'] = pd.to_datetime(data['modification_date'])
    data['publish_date'] = pd.to_datetime(data['publish_date'])
    cs = pd.read_csv('coronavir_stat.csv')
    cs['Дата'] = pd.to_datetime(cs['Дата'])

    data = data.merge(cs, left_on='creation_date', right_on='Дата',how='left')
    data.rename(columns = {'Заражений':'zaraz_creation_date',
                           'Смертей': 'death_creation_date',
                           'Выздоровлений':'vizd_creation_date',
                           'Заражено на дату':'zaraz_on_creation_date'},inplace=True)
    data.drop('Дата',axis=1,inplace=True)

    data = data.merge(cs, left_on='modification_date', right_on='Дата',how='left')
    data.rename(columns = {'Заражений':'zaraz_modification_date',
                           'Смертей': 'death_modification_date',
                           'Выздоровлений':'vizd_modification_date',
                           'Заражено на дату':'zaraz_on_modification_date'},inplace=True)
    data.drop('Дата',axis=1,inplace=True)

    data = data.merge(cs, left_on='publish_date', right_on='Дата',how='left')
    data.rename(columns = {'Заражений':'zaraz_publish_date',
                           'Смертей': 'death_publish_date',
                           'Выздоровлений':'vizd_publish_date',
                           'Заражено на дату':'zaraz_on_publish_date'},inplace=True)
    data.drop('Дата',axis=1,inplace=True)

    data.drop(['creation_date','modification_date','publish_date'],axis=1,inplace=True)
    data = data.fillna(0)
    return data

In [ ]:
data_cs = coronavir_stat(train)
data_test_cs = coronavir_stat(test)

In [ ]:
train = train.merge(data_cs, left_on='id', right_on='id', how='left')
test = test.merge(data_test_cs, left_on='id', right_on='id', how='left')

In [ ]:



train['creation_date'] = pd.to_datetime(train['creation_date'], format='%Y-%m-%d')
train['modification_date'] = pd.to_datetime(train['modification_date'], format='%Y-%m-%d')
today_dt = pd.to_datetime('20.12.2020', format='%d.%m.%Y')
train['start_date'] = train['start_date'].fillna(value = today_dt)
train['start_date'] = pd.to_datetime(train['start_date'], format='%Y-%m-%d', errors = 'coerce')
train['start_year'] = pd.DatetimeIndex(train['start_date']).year
train["start_month"]= pd.DatetimeIndex(train["start_date"]).month
train["start_day"] = pd.DatetimeIndex(train["start_date"]).day
train["start_dt_half_m"] =  train["start_day"].apply(lambda x : 2 if (x > 15) else 1)
train["start_quarter"] =  train["start_month"].apply(lambda x : x // 4);

train['finish_date'] = train['finish_date'].fillna(value = today_dt)
train['finish_date'] = pd.to_datetime(train['finish_date'], format='%Y-%m-%d',  errors = 'coerce')
train['finish_year'] = pd.DatetimeIndex(train['finish_date']).year
train["finish_month"]= pd.DatetimeIndex(train["finish_date"]).month
train["finish_day"] = pd.DatetimeIndex(train["finish_date"]).day
train["finish_dt_half_m"] =  train["finish_day"].apply(lambda x : 2 if (x > 15) else 1)
train["finish_quarter"] =  train["finish_month"].apply(lambda x : x // 4);

lockdown_dt = pd.to_datetime('27.03.2020', format='%d.%m.%Y')
delta = datetime.timedelta(seconds=1)
train['days_bt_lockdown_cr'] =  train['creation_date'].apply(lambda x:max(x - lockdown_dt,  delta).days)
train['days_bt_lockdown_mod'] =  train['modification_date'].apply(lambda x:max(x - lockdown_dt,  delta).days)
train['days_at_last_work'] = (train['finish_date'] - train['start_date'])
train['days_at_last_work'] = train['days_at_last_work'].apply(lambda x: x.days)

In [ ]:
train['graduate_age'] = train['graduation_year'] - (pd.DatetimeIndex(train['modification_date']).year - train['age'])

train['start_date'] = train['start_date'].fillna(value = today_dt)
train['start_date'] = pd.to_datetime(train['start_date'], format='%Y-%m-%d', errors = 'coerce')
train['finish_date'] = train['finish_date'].fillna(value = today_dt)
train['finish_date'] = pd.to_datetime(train['finish_date'], format='%Y-%m-%d',  errors = 'coerce')
train['finish_winter'] = train['finish_date'].apply(lambda x: 1 if ((x.month < 3) or (x.month == 12)) else 0)
train['finish_spring'] = train['finish_date'].apply(lambda x: 1 if ((x.month > 2) and (x.month < 6)) else 0)
train['finish_summer'] = train['finish_date'].apply(lambda x: 1 if ((x.month > 5) and (x.month < 9)) else 0)
train['finish_autumn'] = train['finish_date'].apply(lambda x: 1 if ((x.month > 8) and (x.month < 12)) else 0)

train['start_winter'] = train['start_date'].apply(lambda x: 1 if ((x.month < 3) or (x.month == 12)) else 0)
train['start_spring'] = train['start_date'].apply(lambda x: 1 if ((x.month > 2) and (x.month < 6)) else 0)
train['start_summer'] = train['start_date'].apply(lambda x: 1 if ((x.month > 5) and (x.month < 9)) else 0)
train['start_autumn'] = train['start_date'].apply(lambda x: 1 if ((x.month > 8) and (x.month < 12)) else 0)

In [ ]:
train.to_csv('train_v14.csv', sep = ',', index = False)
test.to_csv('test_v14.csv', sep = ',', index = False)